In [2]:
#Criar um navegador
#importar dados
#para cada item da tabela
#Procurar itens no google shopping 
#verificar se estão no range
#procurar no buscape
#verificar se estão no range
#Salvar as ofertas boas num Dataframe
#exportar para excel
#enviar por email